In [3]:
from deepcave import Recorder, Objective
from deepcave.runs.converters.deepcave import DeepCAVERun
from deepcave.plugins.hyperparameter.importances import Importances
import pandas as pd
from arlbench.core.algorithms import DQN, PPO, SAC
from pathlib import Path
from ConfigSpace import ConfigurationSpace, Float, Categorical, Constant
from collections import OrderedDict
import math
import os

deepcave.utils.styled_plot (WARNING): LaTeX not found. Using default font.


In [4]:
replacement_dict = {"atari_qbert": "Atari", 'atari_double_dunk': "Atari", 'atari_phoenix': "Atari", 'atari_this_game': "Atari", 'atari_battle_zone': "Atari", 
                    'box2d_lunar_lander': "Box2d", "box2d_bipedal_walker": "Box2d", 'cc_acrobot': "CC", 'cc_cartpole': "CC", 'cc_mountain_car': "CC", "cc_pendulum": "CC", 'cc_continuous_mountain_car': "CC",
                    'minigrid_door_key': "XLand", 'minigrid_empty_random': "XLand", 'minigrid_four_rooms': "XLand", 
                    'minigrid_unlock': "XLand", "brax_ant": "Brax", "brax_halfcheetah": "Brax", "brax_hopper": "Brax", "brax_humanoid": "Brax"}
algorithms = {"dqn": DQN, "ppo": PPO, "sac": SAC}
envs = ["atari_qbert", 'atari_double_dunk', 'atari_phoenix', 'atari_this_game', 'atari_battle_zone', 
        'box2d_lunar_lander', 'box2d_bipedal_walker', 'cc_acrobot', 'cc_cartpole', 'cc_mountain_car', 'cc_pendulum', 'cc_continuous_mountain_car',
        'minigrid_door_key', 'minigrid_empty_random', 'minigrid_four_rooms', 'minigrid_unlock', 'brax_ant', 'brax_halfcheetah', 'brax_hopper', 'brax_humanoid']


In [5]:
#  noqa: D400
"""
# fANOVA

This module provides a tool for assessing the importance of an algorithms Hyperparameters.

Utilities provide calculation of the data wrt the budget and train the forest on the encoded data.

## Classes
    - fANOVA: Calculate and provide midpoints and sizes.
"""

from typing import Any, Dict, List, Optional, Tuple, Union

import itertools as it

import numpy as np

from deepcave.constants import COMBINED_COST_NAME
from deepcave.evaluators.epm.fanova_forest import FanovaForest
from deepcave.runs import AbstractRun
from deepcave.runs.objective import Objective


class fANOVA:
    """
    Calculate and provide midpoints and sizes.

    They are generated from the forest's split values in order to get the marginals.

    Properties
    ----------
    run : AbstractRun
        The Abstract Run used for the calculation.
    cs : ConfigurationSpace
        The configuration space of the run.
    hps : List[Hyperparameters]
        The Hyperparameters of the configuration space.
    hp_names : List[str]
        The corresponding names of the Hyperparameters.
    n_trees : int
        The number of trees.
    """

    def __init__(self, run: AbstractRun):
        if run.configspace is None:
            raise RuntimeError("The run needs to be initialized.")

        self.run = run
        self.cs = run.configspace
        self.hps = self.cs.get_hyperparameters()
        self.hp_names = self.cs.get_hyperparameter_names()
        self.n_dims = len(self.hps)

    def calculate(
        self,
        objectives: Optional[Union[Objective, List[Objective]]] = None,
        budget: Optional[Union[int, float]] = None,
        n_trees: int = 16,
        seed: int = 0,
    ) -> None:
        """
        Get the data with respect to budget and train the forest on the encoded data.

        Note
        ----
        Right now, only `n_trees` is used. It can be further specified if needed.

        Parameters
        ----------
        objectives : Optional[Union[Objective, List[Objective]]], optional
            Considered objectives. By default None. If None, all objectives are considered.
        budget : Optional[Union[int, float]], optional
            Considered budget. By default None. If None, the highest budget is chosen.
        n_trees : int, optional
            How many trees should be used. By default 16.
        seed : int
            Random seed. By default 0.
        """
        if objectives is None:
            objectives = self.run.get_objectives()

        if budget is None:
            budget = self.run.get_highest_budget()

        self.n_trees = n_trees

        # Get data
        df = self.run.get_encoded_data(
            objectives, budget, specific=True, include_combined_cost=True
        )
        X = df[self.hp_names].to_numpy()
        # Combined cost name includes the cost of all selected objectives
        Y = df[COMBINED_COST_NAME].to_numpy()

        # Get model and train it
        self._model = FanovaForest(self.cs, n_trees=n_trees, seed=seed)
        self._model.train(X, Y)

    def get_importances(
        self, hp_names: Optional[List[str]] = None, depth: int = 1, sort: bool = True
    ) -> Dict[Union[str, Tuple[str, ...]], Tuple[float, float, float, float]]:
        """
        Return the importance scores from the passed Hyperparameter names.

        Warning
        -------
        Using a depth higher than 1 might take much longer.

        Parameters
        ----------
        hp_names : Optional[List[str]]
            Selected Hyperparameter names to get the importance scores from. If None, all
            Hyperparameters of the configuration space are used.
        depth : int, optional
            How often dimensions should be combined. By default 1.
        sort : bool, optional
            Whether the Hyperparameters should be sorted by importance. By default True.

        Returns
        -------
        Dict[Union[str, Tuple[str, ...]], Tuple[float, float, float, float]]
            Dictionary with Hyperparameter names and the corresponding importance scores.
            The values are tuples of the form (mean individual, var individual, mean total,
            var total). Note that individual and total are the same if depth is 1.

        Raises
        ------
        RuntimeError
            If there is zero total variance in all trees.
        """
        if hp_names is None:
            hp_names = self.cs.get_hyperparameter_names()

        hp_ids = []
        for hp_name in hp_names:
            hp_ids.append(self.cs.get_idx_by_hyperparameter_name(hp_name))

        # Calculate the marginals
        vu_individual, vu_total = self._model.compute_marginals(hp_ids, depth)

        importances: Dict[Tuple[Any, ...], Tuple[float, float, float, float]] = {}
        for k in range(1, len(hp_ids) + 1):
            if k > depth:
                break

            for sub_hp_ids in it.combinations(hp_ids, k):
                sub_hp_ids = tuple(sub_hp_ids)

                # clean here to catch zero variance in a trees
                non_zero_idx = np.nonzero(
                    [self._model.trees_total_variance[t] for t in range(self.n_trees)]
                )

                if len(non_zero_idx[0]) == 0:
                    raise RuntimeError("Encountered zero total variance in all trees.")

                fractions_total = np.array(
                    [
                        vu_total[sub_hp_ids][t] / self._model.trees_total_variance[t]
                        for t in non_zero_idx[0]
                    ]
                )
                fractions_individual = np.array(
                    [
                        vu_individual[sub_hp_ids][t] / self._model.trees_total_variance[t]
                        for t in non_zero_idx[0]
                    ]
                )

                importances[sub_hp_ids] = (
                    np.mean(fractions_individual),
                    np.var(fractions_individual),
                    np.mean(fractions_total),
                    np.var(fractions_total),
                )

        # Sort by total mean fraction
        if sort:
            importances = {
                k: v for k, v in sorted(importances.items(), key=lambda item: item[1][2])
            }

        # The ids get replaced with hyperparameter names again
        all_hp_names = self.cs.get_hyperparameter_names()
        importances_: Dict[Union[str, Tuple[str, ...]], Tuple[float, float, float, float]] = {}
        for hp_ids_importances, values in importances.items():
            hp_names = [all_hp_names[hp_id] for hp_id in hp_ids_importances]
            hp_names_key: Union[Tuple[str, ...], str]
            if len(hp_names) == 1:
                hp_names_key = hp_names[0]
            else:
                hp_names_key = tuple(hp_names)
            importances_[hp_names_key] = values

        return importances_

    def marginal_mean_variance_for_values(self, dimlist, values_to_predict):
        """
        Return the marginal of selected parameters for specific values

        Parameters
        ----------
        dimlist: list
                Contains the indices of ConfigSpace for the selected parameters
                (starts with 0)
        values_to_predict: list
                Contains the values to be predicted

        Returns
        -------
        tuple
            marginal mean prediction and corresponding variance estimate
        """
        sample = np.full(self.n_dims, np.nan, dtype=np.float)
        for i in range(len(dimlist)):
            sample[dimlist[i]] = values_to_predict[i]

        return self._model.forest.marginal_mean_variance_prediction(sample)

    def get_most_important_pairwise_marginals(self, params=None, n=10):
        """
        Return the n most important pairwise marginals from the whole ConfigSpace.

        Parameters
        ----------
        params: list of strings or ints
            If specified, limit analysis to those parameters. If ints, interpreting as indices from
            ConfigurationSpace
        n: int
             The number of most relevant pairwise marginals that will be returned

        Returns
        -------
        list:
             Contains the n most important pairwise marginals
        """
        self.tot_imp_dict = OrderedDict()
        pairwise_marginals = []
        if params is None:
            dimensions = range(self.n_dims)
        else:
            if type(params[0]) == str:
                idx = []
                for i, param in enumerate(params):
                    idx.append(self.cs.get_idx_by_hyperparameter_name(param))
                dimensions = idx

            else:
                dimensions = params
        # pairs = it.combinations(dimensions,2)
        pairs = [x for x in it.combinations(self.hp_names, 2)]
        if params:
            n = len(list(pairs))
        for combi in pairs:
            pairwise_marginal_performance = self.get_importances(combi, depth=2)
            tot_imp = pairwise_marginal_performance[combi][0]
            #combi_names = [self.hps[combi[0]].name, self.hps[combi[1]].name]
            pairwise_marginals.append((tot_imp, combi[0], combi[1]))

        pairwise_marginal_performance = sorted(pairwise_marginals, reverse=True)

        for marginal, p1, p2 in pairwise_marginal_performance[:n]:
            self.tot_imp_dict[(p1, p2)] = marginal

        return self.tot_imp_dict

    def get_triple_marginals(self, params=None):
        """
        Return the n most important pairwise marginals from the whole ConfigSpace

        Parameters
        ----------
        params: list
             The parameters

        Returns
        -------
        list:
             Contains most important triple marginals
        """
        self.tot_imp_dict = OrderedDict()
        triple_marginals = []
        if len(params) < 3:
            raise RuntimeError(
                "Number of parameters have to be greater than %i. At least 3 parameters needed"
                % len(params)
            )
        if type(params[0]) == str:
            idx = []
            for i, param in enumerate(params):
                idx.append(self.cs.get_idx_by_hyperparameter_name(param))
            dimensions = idx

        else:
            dimensions = params

        triplets = [x for x in it.combinations(self.hp_names, 3)]
        for combi in triplets:
            triple_marginal_performance = self.get_importances(combi, depth=3)
            tot_imp = triple_marginal_performance[combi][0]
            combi_names = [
                self.hps[combi[0]].name,
                self.hps[combi[1]].name,
                self.hps[combi[2]].name,
            ]
            triple_marginals.append((tot_imp, combi_names[0], combi_names[1], combi_names[2]))

        triple_marginal_performance = sorted(triple_marginals, reverse=True)
        if params:
            triple_marginal_performance = triple_marginal_performance[: len(list(triplets))]

        for marginal, p1, p2, p3 in triple_marginal_performance:
            self.tot_imp_dict[(p1, p2, p3)] = marginal

        return self.tot_imp_dict

In [6]:
def get_importances(path, algorithm, method="local"):
    # Instantiate the run
    run = DeepCAVERun.from_path(Path(path))
    objective_id = run.get_objective_ids()[0]
    budget_ids = run.get_budget_ids()

    # Instantiate the plugin
    plugin = Importances()

    inputs = plugin.generate_inputs(
        objective_id=objective_id,
        hyperparameter_names= algorithms[algorithm].get_hpo_search_space().get_hyperparameter_names(), 
        budget_ids=budget_ids,
        method=method, n_hps=10, n_trees=10
    )
    # Now we use what we get from deepcave to get the marginals with the full fanova options
    objective = run.get_objective(inputs["objective_id"])
    method = inputs["method"]
    n_trees = inputs["n_trees"]

    hp_dict = run.configspace.get_hyperparameters_dict()
    if method == "global" and any([type(v) == Constant for v in hp_dict.values()]):
            hp_dict_wo_const = {k: v for k, v in hp_dict.items() if type(v) != Constant}
            configspace_wo_const = ConfigurationSpace()
            for k in hp_dict_wo_const.keys():
                configspace_wo_const.add_hyperparameter(hp_dict_wo_const[k])
            configspace_wo_const.add_conditions(run.configspace.get_conditions())
            configspace_wo_const.add_forbidden_clauses(run.configspace.get_forbiddens())
            run.configspace = configspace_wo_const

            configs_wo_const = []
            for n in range(len(run.configs)):
                configs_wo_const.append(
                    {k: v for k, v in run.configs[n].items() if k in hp_dict_wo_const.keys()}
                )
            run.configs = dict(enumerate(configs_wo_const))

    hp_names = run.configspace.get_hyperparameter_names()
    budgets = run.get_budgets(include_combined=True)

    evaluator = fANOVA(run)
    data = {}
    for budget_id, budget in enumerate(budgets):
        #assert isinstance(budget, (int, float))
        evaluator.calculate(objective, budget, n_trees=n_trees, seed=0)
        pairwise_marginals = evaluator.get_most_important_pairwise_marginals(n=5)
    return pairwise_marginals

In [7]:
def get_basic_importances(path, algorithm, method="local"):
    # Instantiate the run
    run = DeepCAVERun.from_path(Path(path))
    objective_id = run.get_objective_ids()[0]
    budget_ids = run.get_budget_ids()

    # Instantiate the plugin
    plugin = Importances()

    inputs = plugin.generate_inputs(
        objective_id=objective_id,
        hyperparameter_names= algorithms[algorithm].get_hpo_search_space().get_hyperparameter_names(), 
        budget_ids=budget_ids,
        method=method, n_hps=10, n_trees=10
    )
    # Now we use what we get from deepcave to get the marginals with the full fanova options
    objective = run.get_objective(inputs["objective_id"])
    method = inputs["method"]
    n_trees = inputs["n_trees"]

    hp_dict = run.configspace.get_hyperparameters_dict()
    if method == "global" and any([type(v) == Constant for v in hp_dict.values()]):
            hp_dict_wo_const = {k: v for k, v in hp_dict.items() if type(v) != Constant}
            configspace_wo_const = ConfigurationSpace()
            for k in hp_dict_wo_const.keys():
                configspace_wo_const.add_hyperparameter(hp_dict_wo_const[k])
            configspace_wo_const.add_conditions(run.configspace.get_conditions())
            configspace_wo_const.add_forbidden_clauses(run.configspace.get_forbiddens())
            run.configspace = configspace_wo_const

            configs_wo_const = []
            for n in range(len(run.configs)):
                configs_wo_const.append(
                    {k: v for k, v in run.configs[n].items() if k in hp_dict_wo_const.keys()}
                )
            run.configs = dict(enumerate(configs_wo_const))

    hp_names = run.configspace.get_hyperparameter_names()
    budgets = run.get_budgets(include_combined=True)

    evaluator = fANOVA(run)
    data = {}
    for budget_id, budget in enumerate(budgets):
        #assert isinstance(budget, (int, float))
        evaluator.calculate(objective, budget, n_trees=n_trees, seed=0)
        importance_data = evaluator.get_importances()
    return importance_data

In [8]:
algorithm = "sac"
deepcave_paths = []
for env in envs:
    if not os.path.isdir(f"deepcave_logs/{algorithm}_{env}"):
        print(f"{env} not found")
    else:
        deepcave_paths.append(f"deepcave_logs/{algorithm}_{env}")
print(deepcave_paths)

atari_qbert not found
atari_double_dunk not found
atari_phoenix not found
atari_this_game not found
atari_battle_zone not found
box2d_lunar_lander not found
cc_acrobot not found
cc_cartpole not found
cc_mountain_car not found
minigrid_door_key not found
minigrid_empty_random not found
minigrid_four_rooms not found
minigrid_unlock not found
['deepcave_logs/sac_box2d_bipedal_walker', 'deepcave_logs/sac_cc_pendulum', 'deepcave_logs/sac_cc_continuous_mountain_car', 'deepcave_logs/sac_brax_ant', 'deepcave_logs/sac_brax_halfcheetah', 'deepcave_logs/sac_brax_hopper', 'deepcave_logs/sac_brax_humanoid']


In [109]:
strings = []
for env in envs:
    try:
        imps = get_importances(Path(f"deepcave_logs/{algorithm}_{env}") / "run_1", algorithm, method="global")
        string = f"{env.replace('_', ' ')}"
        for k in imps.keys():
            value = np.round(imps[k], decimals=2)
            key1, key2 = k
            key1 = key1.replace("_", " ")
            key2 = key2.replace("_", " ")
            string += f" & ({key1}, {key2}) : {value}"
        string += "\\"
        string += "\\"
        print(string)
        strings.append(string)
    except:
        pass

49.59851000000003 501.3891763720703
8.342041972656318 501.3891763720703
70.23471165039066 501.3891763720703
36.545380595703136 501.3891763720703
349.47220898010255 501.3891763720703
43.934540751953136 501.3891763720703
53.776825908203136 501.3891763720703
33.83239841796876 501.3891763720703
7.535584453125011 501.3891763720703
189.16356000000002 501.3891763720703
431.48001 501.3891763720703
447.9184961352539 501.3891763720703
429.4406290087891 501.3891763720703
446.08004856445314 501.3891763720703
435.58758640625 501.3891763720703
444.5686121020508 501.3891763720703
425.1935663256836 501.3891763720703
447.1438150439453 501.3891763720703
440.0080728564453 501.3891763720703
424.28609 501.3891763720703
418.41876 501.3891763720703
418.1996240649414 501.3891763720703
435.5666666064453 501.3891763720703
439.4723215136719 501.3891763720703
436.46328068115236 501.3891763720703
435.08639622070314 501.3891763720703
418.4258432421875 501.3891763720703
382.29906559326173 501.3891763720703
366.30922

In [110]:
for s in strings:
    print(s)
    print("\hline")

box2d bipedal walker & (learning rate, reward scale) : 0.07 & (buffer alpha, learning rate) : 0.03 & (buffer batch size, learning rate) : 0.02 & (learning rate, tau) : 0.02 & (learning rate, learning starts) : 0.02\\
\hline
cc pendulum & (buffer batch size, learning rate) : 0.05 & (buffer alpha, buffer batch size) : 0.03 & (buffer size, learning rate) : 0.02 & (buffer batch size, learning starts) : 0.02 & (learning starts, use target network) : 0.02\\
\hline
cc continuous mountain car & (alpha, alpha auto) : nan & (alpha, buffer alpha) : nan & (alpha, buffer batch size) : nan & (alpha, buffer beta) : nan & (alpha, buffer epsilon) : nan\\
\hline
brax ant & (learning rate, tau) : 0.1 & (learning rate, reward scale) : 0.08 & (buffer size, learning rate) : 0.01 & (learning rate, learning starts) : 0.01 & (buffer alpha, learning rate) : 0.01\\
\hline
brax halfcheetah & (buffer batch size, buffer size) : 0.04 & (buffer alpha, buffer prio sampling) : 0.02 & (learning rate, learning starts) : 

2749.158203125 8329.3486328125
2229.27880859375 8329.3486328125
2196.789794921875 8329.3486328125
2263.206298828125 8329.3486328125
2139.01611328125 8329.3486328125
2716.119384765625 8329.3486328125
2342.643310546875 8329.3486328125
2065.42919921875 8329.3486328125
2274.650390625 8329.3486328125
2387.548095703125 8329.3486328125
5645.631881713867 8329.3486328125
5665.687938690186 8329.3486328125
5729.202884674072 8329.3486328125
5732.41014289856 8329.3486328125
5626.420013427734 8329.3486328125
5475.886627197266 8329.3486328125
5680.643367767334 8329.3486328125
5510.540954589844 8329.3486328125
5565.192611694336 8329.3486328125
5676.207057952881 8329.3486328125
1209.87548828125 8329.3486328125
3383.384765625 8329.3486328125
1741.5498046875 8329.3486328125
776.7314453125 8329.3486328125
1012.94482421875 8329.3486328125
3359.55224609375 8329.3486328125
893.466796875 8329.3486328125
866.63037109375 8329.3486328125
1730.046630859375 8329.3486328125
1563.9921875 8329.3486328125
1590.7998046

In [24]:
dfs = []
for algorithm in ["dqn", "ppo", "sac"]:
    for env in envs:
        print(algorithm, env)
        try:
            img = get_basic_importances(Path(f"deepcave_logs/{algorithm}_{env}") / "run_1", algorithm, method="global")
            mean_fractions = {}
            for k in img.keys():
                value = img[k][0]
                mean_fractions[k] = value
            mean_fractions = dict(sorted(mean_fractions.items(), key=lambda x: x[1], reverse=True)[:2])
            result = {}
            result["algorithm"] = algorithm
            result["env"] = env
            result["top_hp_1"] = list(mean_fractions.keys())[0]
            result["top_hp_2"] = list(mean_fractions.keys())[1]
            df = pd.DataFrame([result])
            dfs.append(df)
        except Exception as e:
            print(e)
df = pd.concat(dfs)
df.to_csv(f"top_2_importances.csv", index=False)

dqn atari_qbert
16975.781 17125.781
13866.2107 17125.781
12768.945 17125.781
14189.843499999999 17125.781
13590.2341 17125.781
16257.812249999999 17125.781
16542.9685 17125.781
16345.11694 17125.781
12672.2654 17125.781
14731.640399999998 17125.781
16655.6638 17125.781
16554.68725 17125.781
16525.781 17125.781
16620.8982 17125.781
16776.171619999997 17125.781
16807.2263 17125.781
16381.249749999999 17125.781
16769.921619999997 17125.781
16731.64038 17125.781
16652.3435 17125.781
17000.781 17125.781
17125.781 17125.781
17000.781 17125.781
16975.781 17125.781
17000.781 17125.781
16975.781 17125.781
16975.781 17125.781
17000.781 17125.781
16975.781 17125.781
17000.781 17125.781
9391.796599999998 17125.781
13071.288799999998 17125.781
7953.905999999999 17125.781
10120.3122 17125.781
8910.155999999999 17125.781
12510.351299999998 17125.781
12837.304399999999 17125.781
11791.405999999999 17125.781
9109.374799999998 17125.781
7160.155999999999 17125.781
17125.781 17125.781
17125.781 17125.781

In [23]:
print(dfs[0])

  algorithm          env       top_hp_1  top_hp_2
0       ppo  atari_qbert  learning_rate  clip_eps
